In [1]:
import gym
import torch as th
import numpy as np
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate
import matplotlib.pyplot as plt


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#wrapper for cont env with PPO, plot result for several steps
def ppo_eval_interval(p, L, t_t, n_iter, n_step, gae, learning_rate=0.0003):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1, 'action':20.0}
    PolicyCONFIG = dict(activation_fn=th.nn.Tanh,
                     net_arch=[dict(pi=[64,64,64], vf=[64,64,64])])
    cont_env = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=0, gamma = 1, gae_lambda=gae, n_epochs=16,
                     learning_rate = learning_rate,use_sde = False, n_steps = n_step, policy_kwargs = PolicyCONFIG)
#     cont_model = PPO(MlpPolicy, cont_env, verbose=0, gamma = 1, gae_lambda=gae,
#                    learning_rate = learning_rate,use_sde = False, n_steps = n_step)
    env_eval = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    env_eval = make_vec_env('inventory_cont_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=2*4*n_step-1)#each iteration has 8192 timesteps with n_env=4
        timesteps = timesteps + 2*4*n_step

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(-res_mean)
        res_std_arr.append(res_std)
        
        if -res_mean == min(res_mean_arr):
            cont_model.save("ppo_min_model_"+str(p)+"_"+str(L))
        
#         print(res_mean_arr)
    
    plt.plot(res_mean_arr)
    plt.xlabel("Iteration")
    plt.ylabel("Average cost")
    plt.title("L="+str(L)+", p="+str(p))
    plt.show()
    
    min_model = PPO.load("ppo_min_model_"+str(p)+"_"+str(L))
    mean_min, std_min = ppo_evaluate(min_model, env_eval, 50000)
    print("p="+str(p)+"， L="+str(L)+": mean "+str(-mean_min)+", std_dev: "+str(std_min))
    
    return res_mean_arr, res_std_arr, mean_min, std_min

In [4]:
# listp = [0.25,1,4,9,39,99]
# listL = [4,70]
listp = [99]
listL = [100]
# t_t = 100000
n_iter = 500
gae_lambda = 0.95
# n_step = 8192
learning_rate = 0.0003
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        n_step = 1600
        t_t = 800*4*n_step
        res_mean, res_std, mean_min, std_min = ppo_eval_interval(p,L,t_t, n_iter, n_step, gae_lambda, learning_rate)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':-mean_min, 'res_std': std_min}, ignore_index=True)

Running PPO w/: p= 99 , L= 100
mean:  -68.61220061832611
standard deviation: 5.722372786020594
mean:  -60.60675037993472
standard deviation: 1.7579605168542913
mean:  -65.74868480366506
standard deviation: 4.508411152665166
mean:  -68.6205304091896
standard deviation: 3.945675588199331
mean:  -71.60532395038945
standard deviation: 4.736066046771128
mean:  -72.63133422770265
standard deviation: 3.846185403080878
mean:  -70.89809375734451
standard deviation: 7.398663670179564
mean:  -71.60569688626774
standard deviation: 4.108245893199715
mean:  -73.57279385913583
standard deviation: 1.7457013168144297
mean:  -74.78115921829192
standard deviation: 2.81287950240461
mean:  -75.79569191560711
standard deviation: 3.0625373575325563
mean:  -75.833429831427
standard deviation: 4.491536430851112
mean:  -69.19426359655901
standard deviation: 2.990503992962308
mean:  -74.89671661759671
standard deviation: 4.230992139062081
mean:  -69.79843038376659
standard deviation: 4.758789722359917
mean:  -74

mean:  -99.78338833896817
standard deviation: 3.119722831916506
mean:  -100.1095684564352
standard deviation: 7.540604448888806
mean:  -94.91285573866926
standard deviation: 2.6095760626057958
mean:  -95.6326331270121
standard deviation: 1.5239219102817327
mean:  -97.18663368157671
standard deviation: 4.288729970615649
mean:  -102.02571413252204
standard deviation: 6.9357335009419785
mean:  -103.99059211919457
standard deviation: 2.1938577285848653
mean:  -97.79301793382913
standard deviation: 3.113942251698429
mean:  -100.65741802076697
standard deviation: 4.309158717068385
mean:  -100.75223187265993
standard deviation: 2.8272725350716716
mean:  -98.38751437246239
standard deviation: 4.813169620363082
mean:  -98.09532692788318
standard deviation: 3.145400263431844
mean:  -99.34429399184584
standard deviation: 4.498637144265328
mean:  -101.62049398373217
standard deviation: 2.4895709593355204
mean:  -96.37140569771229
standard deviation: 4.379211085996243
mean:  -103.48887638355605
sta

KeyboardInterrupt: 

In [ ]:
ppo_res

In [ ]:
plt.plot(res_mean)

## Below is obsolete.